<a href="https://colab.research.google.com/github/mariasald/avd-2024-01/blob/main/Atividade_Pr%C3%A1tica_2_Gera%C3%A7%C3%A3o_de_VA_e_fila_M_M_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Alunos

* Gabriel César Tavares Ferreira - 21854868
* Maria Luiza Saldanha - 22153140

#Bibliotecas Utilizadas

In [2]:
import math
import numpy as np
import random

# 1. Implementar em Python uma VA Exponencial chamada X, com média β=1/9 utilizando o gerador de N.A. do Python.

VA Exponencial com média β
 * o Gerar U ~ U[0, 1]
 * o Retornar $X = -β \ ln (1 – U)$


In [28]:
def va_exponencial(media):
  u = random.random()
  x = -media * math.log (1 - u)
  return round(x,3)

print(va_exponencial(1/9)) #nao esquecer de ver quando receber taxa e quando receber media!!!!!

0.002


Primeiros 10 resultados para média β = 1/9 (taxa = 9):

0.023,
0.0,
0.156,
0.078,
0.016,
0.014,
0.435,
0.027,
0.077,
0.002

# 2. Implementar em Python uma VA Erlang chamada Y, com 4 estágios e média β=1/9 utilizando o gerador de N.A. do Python.

VA Erlang com k estágios e média λ
 * o Gerar Ui ~ U[0, 1], i= 1,..., k
 * o Retornar $X=\frac{-\lambda}{k}ln(\displaystyle\prod_{i=1}^k U^i)$

In [41]:
def va_erlang(media, estagios):
  Ui = [random.random() for _ in range(estagios)]
  produto_Ui = 1.0
  for numero in Ui:
    produto_Ui *= numero

  x = -media * estagios * math.log(produto_Ui)

  return round(x,3)

print(va_erlang(1/9, 4))

2.034


Primeiros 10 resultados para média β = 1/9 (taxa = 9) e 4 estágios:

1.479,
2.23,
1.504,
0.824,
1.268,
2.888,
3.004,
1.032,
3.98,
2.034.

# 3. Gerar a função cumulativa de distribuição F(x) e F(y).

# 4. Plotar F(x) e F(y).

# 5. Implementar em Python uma fila M/M/1.
  * taxa de entrada: λ=9 clientes por segundo
  * taxa de serviço: μ=10 clientes por segundo.


# 6. Executar a simulação da fila M/M/1 gerando $n$ clientes:

* $n=10^3,n=10^6,n=10^9,n=10^{12}$


# 7. Para cada valor de $n$

 * Calcular o tempo de espera $x_{i}$ na fila para cada cliente $i$


 * Estimar o tempo médio $\bar{X}(n)$ de espera na fila M/M/1 e um intervalo de confiança de 95%.

 * Comparar com o valor esperado. Da Teoria das Filas, esse valor é dado por $E[X]=\rho \frac{1/\mu}{1-\rho}$, onde $p=\frac{\lambda}{\mu}$